# 2020 SUMMER ESC :: Week 1 세션 과제

2020.07.26 김채형

### Q1) 이 경우에서는 Type 1 error 와 Type 2 Error 중 무엇이 더 높을까요? FP, FN과 관련지어 설명해주세요

위 경우에서는 threshold가 낮아 FN이 낮고 FP가 높다. 이때 FN은 Type 1 error에 대응되고 FP는 Type 2 error에 대응되기 때문에 결과적으로 Type 1 error에 비해 Type 2 error가 더 높다고 볼 수 있다.

### Q2-1) Accuracy, Precision and Recall이 무엇인지 TP, FP, TN, FN 의 식으로 나타내 주세요.

Accuracy = (TP + TN) / (TP + TN + FP + FN)  
Precision = TP / (TP + FP)  
Recall = TP / (TP + FN)  

### Q2-2) Precision & Recall 에 관한 예시 하나를 들어주세요.

FP의 위험성이 높은 분야에서는 Precision을 우선적으로, FN의 위험성이 높은 분야에서는 Recall을 우선적으로 사용하는 것이 일반적이다. 예를 들어, 우리가 사법 기관에서 무죄추정의 원칙을 기본으로 하는 것은 유죄인 사람이 무죄를 받는 것보다 무죄인 사람이 유죄를 받는 것에 더 초점을 맞추어 이를 방지하고자 하기 위함이다. 이러한 경우에는 Precision을 우선시 한다고 볼 수 있다. 반면 음주 운전 단속의 경우에는 실제로 음주 운전을 했는데 안 했다고 하는 것이 보다 더 위험하기 때문에 Recall을 우선적으로 고려할 필요가 있다.

### Q3) 코드 따라해보고 주석 달기! 

[코드 출처](https://www.geeksforgeeks.org/identifying-handwritten-digits-using-logistic-regression-pytorch/)

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root = './data',
                           train = True,
                           transform = transforms.ToTensor(),
                           download = True)
test_dataset = dsets.MNIST(root = './data',
                          train = False,
                           transform = transforms.ToTensor())

In [3]:
# Hyper Parameters
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# Dataset Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size, 
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [5]:
# Define our model
class LogisticRegression(nn.Module):
    
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [6]:
model = LogisticRegression(input_size, num_classes)

In [7]:
# Set our loss function & optimiser
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [8]:
# Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # 1. Reset all gradients to 0
        outputs = model(images) # 2. Make a forward pass
        loss = criterion(outputs, labels) # 3. Calculate the loss
        loss.backward() # 4. Perform backpropagation
        optimizer.step() # 5. Update all weights
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_dataset) // batch_size, loss.data)) 

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.2049
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1466
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.9971
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9440
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8824
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.8366
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.8027
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.7118
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.5711
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5623
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5053
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.3734
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.4168
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4881
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3270
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2609
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2020
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2369
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2416
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1919
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [9]:
# Test the model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted==labels).sum()
print('Accuracy of the model on the 10000 test images: % d %%'
      % (100*correct/total))

Accuracy of the model on the 10000 test images:  82 %


/Users/distiller/project/conda/conda-bld/pytorch_1591914925853/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


### Q4-1) 2-4 코드에서 optim.SGD를 사용하지않고 코드를 짠다면 어떤 방식으로 짜야할지 설명해주세요.

In [10]:
import torch
from torch import nn
from torch import tensor

In [11]:
# Generate dataset
x_data = tensor([[1.0], [2.0], [3.0]])
y_data = tensor([[2.0], [4.0], [6.0]])
w = torch.tensor([1.0], requires_grad=True)

# Define forward pass
def forward(x):
    return x*w

# Define loss function
def loss(y_pred, y_val):
    return (y_pred-y_val)**2

# Training loop
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    for x_val, y_val in zip(x_data, y_data):
        
        y_pred = forward(x_val) # Forward pass
        l = loss(y_pred, y_val) # Loss
        l.backward() # Back propagation
        
        print('\tgrad: ', x_val, y_val, w.grad.item())
        
        w.data = w.data - 0.01*w.grad.item() # w 업데이트
        w.grad.data.zero_() # w 초기화

    print(f'Epoch: {epoch} | Loss: {l.item()}')

	grad:  tensor([1.]) tensor([2.]) -2.0
	grad:  tensor([2.]) tensor([4.]) -7.840000152587891
	grad:  tensor([3.]) tensor([6.]) -16.228801727294922
Epoch: 0 | Loss: 7.315943717956543
	grad:  tensor([1.]) tensor([2.]) -1.478623867034912
	grad:  tensor([2.]) tensor([4.]) -5.796205520629883
	grad:  tensor([3.]) tensor([6.]) -11.998146057128906
Epoch: 1 | Loss: 3.9987640380859375
	grad:  tensor([1.]) tensor([2.]) -1.0931644439697266
	grad:  tensor([2.]) tensor([4.]) -4.285204887390137
	grad:  tensor([3.]) tensor([6.]) -8.870372772216797
Epoch: 2 | Loss: 2.1856532096862793
	grad:  tensor([1.]) tensor([2.]) -0.8081896305084229
	grad:  tensor([2.]) tensor([4.]) -3.1681032180786133
	grad:  tensor([3.]) tensor([6.]) -6.557973861694336
Epoch: 3 | Loss: 1.1946394443511963
	grad:  tensor([1.]) tensor([2.]) -0.5975041389465332
	grad:  tensor([2.]) tensor([4.]) -2.3422164916992188
	grad:  tensor([3.]) tensor([6.]) -4.848389625549316
Epoch: 4 | Loss: 0.6529689431190491
	grad:  tensor([1.]) tensor([2.])

	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 59 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 60 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 61 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 62 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06


	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 112 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 113 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 114 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 115 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 116 | Loss: 9.094947017729282e-13
	grad:  tens

	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 159 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 160 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 161 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 162 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 163 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  ten

	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 216 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 217 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 218 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 219 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 220 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  ten

...

	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 777 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 778 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 779 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 780 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e

	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 845 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 846 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 847 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 848 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 849 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  ten

	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 886 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 887 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 888 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 889 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 890 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  ten

	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 943 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 944 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 945 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 946 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 947 | Loss: 9.094947017729282e-13
	grad:  tens

	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 983 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 984 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 985 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 986 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  tensor([2.]) tensor([4.]) -2.86102294921875e-06
	grad:  tensor([3.]) tensor([6.]) -5.7220458984375e-06
Epoch: 987 | Loss: 9.094947017729282e-13
	grad:  tensor([1.]) tensor([2.]) -7.152557373046875e-07
	grad:  ten

### Q4-2) Gradient descent 코드를 구현하는 문제입니다. 2-4의 코드에서 다른 optimizer 3개를 이용하여 결과값을 비교해주세요.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

**RMSprop**

In [13]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.193, b: 0.788 Cost: 4.714807
Epoch  100/1000 W: 0.518, b: 1.063 Cost: 0.165974
Epoch  200/1000 W: 0.694, b: 0.678 Cost: 0.067547
Epoch  300/1000 W: 0.850, b: 0.333 Cost: 0.016410
Epoch  400/1000 W: 0.952, b: 0.106 Cost: 0.001692
Epoch  500/1000 W: 0.993, b: 0.016 Cost: 0.000040
Epoch  600/1000 W: 1.000, b: 0.001 Cost: 0.000000
Epoch  700/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  800/1000 W: 1.000, b: 0.000 Cost: 0.000000
Epoch  900/1000 W: 1.007, b: 0.007 Cost: 0.000265
Epoch 1000/1000 W: 1.002, b: 0.002 Cost: 0.000043


**Adam**

In [14]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.053, b: 0.846 Cost: 2.420716
Epoch  100/1000 W: 0.461, b: 1.204 Cost: 0.210404
Epoch  200/1000 W: 0.569, b: 0.952 Cost: 0.132647
Epoch  300/1000 W: 0.682, b: 0.702 Cost: 0.072211
Epoch  400/1000 W: 0.780, b: 0.485 Cost: 0.034513
Epoch  500/1000 W: 0.857, b: 0.315 Cost: 0.014580
Epoch  600/1000 W: 0.913, b: 0.193 Cost: 0.005456
Epoch  700/1000 W: 0.950, b: 0.111 Cost: 0.001809
Epoch  800/1000 W: 0.973, b: 0.060 Cost: 0.000530
Epoch  900/1000 W: 0.986, b: 0.030 Cost: 0.000137
Epoch 1000/1000 W: 0.993, b: 0.015 Cost: 0.000031


**Adadelta**

In [15]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model = LinearRegressionModel()

# optimizer 설정
optimizer = optim.Adadelta(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.208, b: 0.715 Cost: 3.867588
Epoch  100/1000 W: -0.204, b: 0.719 Cost: 3.819863
Epoch  200/1000 W: -0.199, b: 0.724 Cost: 3.759923
Epoch  300/1000 W: -0.193, b: 0.730 Cost: 3.690643
Epoch  400/1000 W: -0.186, b: 0.737 Cost: 3.613771
Epoch  500/1000 W: -0.179, b: 0.744 Cost: 3.530621
Epoch  600/1000 W: -0.171, b: 0.752 Cost: 3.442243
Epoch  700/1000 W: -0.163, b: 0.761 Cost: 3.349527
Epoch  800/1000 W: -0.154, b: 0.769 Cost: 3.253246
Epoch  900/1000 W: -0.144, b: 0.779 Cost: 3.154080
Epoch 1000/1000 W: -0.135, b: 0.788 Cost: 3.052647
